In [25]:
# import needed packages
import pandas as pd
import string

In [26]:
# load fish data
fish = pd.read_csv('../original_data/FISH.csv')
# fish dataframe
fish.head(3)

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity
0,Low4,42.286604,-83.475661,8/25/2015,Gizzard shad,1
1,Low9,42.283461,-83.505311,8/22/2016,Gizzard shad,9
2,Low11,42.297215,-83.525437,8/22/2016,Gizzard shad,11


In [27]:
# Capitalize common name words for easier merging
fish['Common Name'] = fish['Common Name'].apply(lambda x: string.capwords(x))

In [30]:
# load species data
species = pd.read_csv('../P51_SPECIES.csv')
# species dataframe
species.head(3)

,Common Name,Category
0,Sea Lamprey,Intolerant
1,Silver Lamprey,Intolerant
2,Northern Brook Lamprey,Intolerant


In [31]:
# merge data
combined = fish.merge(species, how='left', on='Common Name')

In [32]:
# merged dataframe, now containing fish info and category
combined.head(3)

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity,Category
0,Low4,42.286604,-83.475661,8/25/2015,Gizzard Shad,1,NaN
1,Low9,42.283461,-83.505311,8/22/2016,Gizzard Shad,9,NaN
2,Low11,42.297215,-83.525437,8/22/2016,Gizzard Shad,11,NaN


In [22]:
# save to new csv 
####### NEED TO FIGURE OUT HOW TO SAVE IN DIFF FOLDER #######
combined.to_csv('SPECIES_FISH.csv')  

In [33]:
# Next, reorganize combined dataframe for specific visualizations

In [66]:
combined.head(3)

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity,Category
0,Low4,42.286604,-83.475661,8/25/2015,Gizzard Shad,1,NaN
1,Low9,42.283461,-83.505311,8/22/2016,Gizzard Shad,9,NaN
2,Low11,42.297215,-83.525437,8/22/2016,Gizzard Shad,11,NaN


In [67]:
# group dataframe by field and count number of species
species_per_field = combined.groupby(['FIELDID'], as_index=False)['Common Name'].nunique()
species_per_field.head(3)

,FIELDID,Common Name
0,Bell1,10
1,Bell4,10
2,Bish2,11


In [68]:
# save to new csv 
####### NEED TO FIGURE OUT HOW TO SAVE IN DIFF FOLDER #######
combined.to_csv('SPECIES_PER_FIELD.csv')  